In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install wandb -qU
%pip install --no-deps scikeras
%pip install tensorflow-addons

In [ ]:
#imports
import wandb
from wandb.keras import WandbCallback
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
import sklearn
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
from sklearn.metrics import roc_auc_score
import tensorflow_addons as tfa
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import tensorflow
from keras.optimizers import SGD
from keras.layers import Dropout

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


**Data processing **

In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')

# if gpus:
#     try:
#         # Use the first GPU
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)
# else:
#     print('No GPU detected')

1 Physical GPUs, 1 Logical GPU


In [ ]:
#Upload Real Datasets
df_real1 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Blue Order 1 (Ready_4ML).csv')
df_real2 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Red Order 1 (REady_4ML).csv')
df_real3 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/White Order 1 (Ready_4ML).csv')

df_real1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df_real2.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df_real3.rename(columns={'Run Time ': 'Run Time'}, inplace=True)


df2 = df_real1.append(df_real2.append(df_real3))

<ipython-input-4-219118522143>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df_real1.append(df_real2.append(df_real3))


In [ ]:
#Upload Simulated Dataset
df1 = pd.read_csv('/content/drive/MyDrive/Zach/mergedata/RWB_SimDataCombined.csv')

# Rename the 'B' column to 'C' in-place
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)

df1['type'] = 0 #SimData
df2['type'] = 1 #RealData


In [ ]:
#combining sim and real data into one dataset (SKIP IF USING THIELS INEFFICIENCY COEFFICENT -- TIC)
df_combined = df1.append(df2)
df_combined

<ipython-input-6-db18f19e4a07>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df1.append(df2)


,Run Time,msg,node,topics,type
0,0.000000,Published Order,/Order,"/new_order, /rosout",0
1,0.235499,VGR got something,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",0
2,0.288815,VGR is Moving to HBW,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",0
3,0.300417,HBW Recieved an Order,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",0
4,18.445745,HBW Picked Up Material,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",0
...,...,...,...,...,...
33,18.463000,HBW Picked Up Material,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",1
34,0.265810,VGR is Moving to HBW,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",1
35,0.254050,VGR got something,/vgr_node,"/diagnostics, /new_order, /provide_empty_loc, ...",1
36,0.248460,HBW Received an Order,/hbw_node,"/diagnostics, /hbw_pub, /new_order, /rosout, /...",1


In [ ]:
df_real = df2
df_sim = df1


df_real['topics'] = df_real['topics'].apply(lambda x: x.replace(" ", ""))
#print(df_real)
df_sim['topics'] = df_sim['topics'].apply(lambda x: x.replace(" ", ""))
#print(df_sim)

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df_real['topics']= label_encoder.fit_transform(df_real['topics'])
df_real['msg']= label_encoder.fit_transform(df_real['msg'])
df_real['node']= label_encoder.fit_transform(df_real['node'])
#print(df_real)

df_sim['topics']= label_encoder.fit_transform(df_sim['topics'])
df_sim['msg']= label_encoder.fit_transform(df_sim['msg'])
df_sim['node']= label_encoder.fit_transform(df_sim['node'])
#print(df_sim)

actual_data = df_real.drop('type', axis=1)
ideal_data = df_sim.drop('type', axis=1)

print(actual_data.shape)
print(ideal_data.shape)


#TIC = 0 --> Perfect equality between datasets
#0<TIC<1 --> Partial inequality (some groups recieve more resources than others)
#TIC = 1 --> Perfect inequality (one group recieves all the resources)

def theils_inefficiency_coefficient(actual_data, ideal_data):
    # Add a small value (epsilon) to avoid zero values
    epsilon = 1e-9
    actual_data += epsilon
    ideal_data += epsilon

    # Calculate the Theil's Inefficiency Coefficient
    X_actual = np.sum(actual_data)
    proportions_actual = actual_data / X_actual
    log_proportions_actual = np.log(proportions_actual)

    X_ideal = np.sum(ideal_data)
    proportions_ideal = ideal_data / X_ideal
    log_proportions_ideal = np.log(proportions_ideal)

    return np.mean(proportions_actual * (log_proportions_actual - log_proportions_ideal), axis=0)


tic_value = theils_inefficiency_coefficient(actual_data, ideal_data)
print("Theil's Inefficiency Coefficient (TIC) between simulated and actual data:",'\n\n', tic_value)



(114, 4)
(114, 4)
Theil's Inefficiency Coefficient (TIC) between simulated and actual data: 

 Run Time    0.028500
msg        -0.000077
node        0.005113
topics      0.050285
dtype: float64


In [ ]:
df_combined['topics'] = df_combined['topics'].apply(lambda x: x.replace(" ", ""))
df_combined

,Run Time,msg,node,topics,type
0,0.000000,Published Order,/Order,"/new_order,/rosout",0
1,0.235499,VGR got something,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",0
2,0.288815,VGR is Moving to HBW,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",0
3,0.300417,HBW Recieved an Order,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",0
4,18.445745,HBW Picked Up Material,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",0
...,...,...,...,...,...
33,18.463000,HBW Picked Up Material,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",1
34,0.265810,VGR is Moving to HBW,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",1
35,0.254050,VGR got something,/vgr_node,"/diagnostics,/new_order,/provide_empty_loc,/pr...",1
36,0.248460,HBW Received an Order,/hbw_node,"/diagnostics,/hbw_pub,/new_order,/rosout,/vgr_...",1


In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df_combined['topics']= label_encoder.fit_transform(df_combined['topics'])
df_combined['msg']= label_encoder.fit_transform(df_combined['msg'])
df_combined['node']= label_encoder.fit_transform(df_combined['node'])
df_combined

,Run Time,msg,node,topics,type
0,0.000000,25,0,4,0
1,0.235499,37,7,2,0
2,0.288815,39,7,2,0
3,0.300417,21,1,0,0
4,18.445745,14,1,0,0
...,...,...,...,...,...
33,18.463000,14,1,0,1
34,0.265810,39,7,2,1
35,0.254050,37,7,2,1
36,0.248460,20,1,0,1


In [ ]:
#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df_combined.drop(predict, axis=1)
y = df_combined[predict]
print(X.head())
print(y.head())

    Run Time  msg  node  topics
0   0.000000   25     0       4
1   0.235499   37     7       2
2   0.288815   39     7       2
3   0.300417   21     1       0
4  18.445745   14     1       0
0    0
1    0
2    0
3    0
4    0
Name: type, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.3, shuffle=True)

print(f'Train y: {y_train.shape}')
print(f'Train X: {X_train.shape}')
print(f'Test y: {y_test.shape}')
print(f'Test X: {X_test.shape}')

Train y: (159,)
Train X: (159, 4)
Test y: (69,)
Test X: (69, 4)


In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Example sweep configuration
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize',
        'name': 'f1_score'
		},
    'parameters': {
        'batch_size': {'values': [10,15,25,32,50,64,75,100]},
        'epochs': {'values': [50,75,100,125,150,200,250,300]},
        'fc_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'sc_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'Td_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'Frth_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'Fif_layer_size': {'values': [7,9,11,13,15,20,25,30,50]},
        'lr': {'max': 0.1, 'min': 0.0001},
        'optimizer':{'values': ['adam','SGD','RMSprop','nadam','Adadelta','Adagrad','Adamax','Ftrl','AdamW']},
        'dropout': {'values': [0, 0.3, 0.5]},
     },

}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="NN_Turing", entity="dt_zach")

Create sweep with ID: uj0vi50d
Sweep URL: https://wandb.ai/dt_zach/NN_Turing/sweeps/uj0vi50d


In [ ]:
def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
    if optimizer.lower() == "rmsprop":
        return tf.keras.optimizers.RMSprop(learning_rate=lr)
    if optimizer.lower() == "nadam":
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    if optimizer.lower() == "adadelta":
        return tf.keras.optimizers.Adadelta(learning_rate=lr)
    if optimizer.lower() == "adagrad":
        return tf.keras.optimizers.Adagrad(learning_rate=lr)
    if optimizer.lower() == "adamax":
        return tf.keras.optimizers.Adamax(learning_rate=lr)
    if optimizer.lower() == "ftrl":
        return tf.keras.optimizers.Ftrl(learning_rate=lr)
    if optimizer.lower() == "adamw":
        return tf.keras.optimizers.AdamW(learning_rate=lr)

In [ ]:
def Nerual_Net(fc_layer_size=11,sc_layer_size=7,Td_layer_size=7,Frth_layer_size=7,Fif_layer_size=7,dropout=0):
  # Set the GPU as the backend for Keras
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)
  # initiate Network
  network = Sequential()

  # input layer that is fully connected with ReLU activation
  network.add(Dense(5, input_dim=4, activation="relu",use_bias=True,bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(dropout))

  #hiddent layers
  network.add(Dense(fc_layer_size, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(dropout))
  # 2nd hiddent layers
  network.add(Dense(sc_layer_size, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(dropout))
  # 3rd hiddent layers
  network.add(Dense(Td_layer_size, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(dropout))
  # 4th hidden layers
  network.add(Dense(Frth_layer_size, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(dropout))
  #fifth hidden layers
  network.add(Dense(Fif_layer_size, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))


  #Output layer that is fully connected with sigmoid activation
  network.add(Dense(1, activation="sigmoid",kernel_initializer="glorot_normal"))


  #return completed network
  return network

In [ ]:
def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(optimizer=get_optimizer(lr, optimizer) ,
               loss="binary_crossentropy",
               metrics=["binary_accuracy",
               tfa.metrics.F1Score(num_classes=2,
                                  average='macro',
                                  threshold=0.5),
               tf.keras.metrics.AUC(from_logits=True)
                        ])

    model.fit(X_train,
              y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=[WandbCallback()])

In [ ]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "TestLabfab"
        wandb.config.dataset_name = "Labfab"

        # initialize model
        model = Nerual_Net(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.Frth_layer_size,wandb.config.Fif_layer_size,wandb.config.dropout)

        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.lr,
              wandb.config.optimizer)

In [ ]:
import pprint
pprint.pprint(sweep_configuration)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'name': 'sweep',
 'parameters': {'Fif_layer_size': {'values': [7,
                                              9,
                                              11,
                                              13,
                                              15,
                                              20,
                                              25,
                                              30,
                                              50]},
                'Frth_layer_size': {'values': [7,
                                               9,
                                               11,
                                               13,
                                               15,
                                               20,
                                               25,
                                               30,
                                               50]},


In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=150)

wandb: Agent Starting Run: 10uvmc2c with config:
wandb: 	Fif_layer_size: 9
wandb: 	Frth_layer_size: 7
wandb: 	Td_layer_size: 25
wandb: 	batch_size: 25
wandb: 	dropout: 0
wandb: 	epochs: 250
wandb: 	fc_layer_size: 15
wandb: 	lr: 0.07255931426452597
wandb: 	optimizer: Adagrad
wandb: 	sc_layer_size: 13
wandb: Currently logged in as: jbinwu39 (dt_zach). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/250


Run 10uvmc2c errored: ValueError('in user code:\n\n    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1284, in train_function  *\n        return step_function(self, iterator)\n    File "/usr/local/lib/python3.10/dist-packages/tensorflow_addons/metrics/f_scores.py", line 160, in update_state  *\n        self.true_positives.assign_add(_weighted_sum(y_pred * y_true, sample_weight))\n\n    ValueError: Dimension 0 in both shapes must be equal, but are 2 and 1. Shapes are [2] and [1]. for \'{{node AssignAddVariableOp_5}} = AssignAddVariableOp[dtype=DT_FLOAT](AssignAddVariableOp_5/resource, Sum_3)\' with input shapes: [], [1].\n')
wandb: ERROR Run 10uvmc2c errored: ValueError('in user code:\n\n    File "/usr/local/lib/python3.10/dist-packages/keras/engine/training.py", line 1284, in train_function  *\n        return step_function(self, iterator)\n    File "/usr/local/lib/python3.10/dist-packages/tensorflow_addons/metrics/f_scores.py", line 160, in update_state  

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
run = wandb.init(project="NN_Turing", entity="dt_zach")

Error in callback <function _WandbInit._resume_backend at 0x7949a82a8430> (for pre_run_cell):


BrokenPipeError: ignored

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Error in callback <function _WandbInit._pause_backend at 0x7949a82a85e0> (for post_run_cell):


BrokenPipeError: ignored

Testing the best model from Wandb Sweep

In [ ]:
#Build the model
def Neural_Net():

  # initiate Network
  network = Sequential()

  # input layer that is fully connected with ReLU activation
  network.add(Dense(4, input_dim=4, activation="relu",use_bias=True,bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(0))
  #hidden layers
  network.add(Dense(30, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(0))
  network.add(Dense(15, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(0))
  network.add(Dense(25, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(0))
  network.add(Dense(20, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))
  network.add(Dropout(0))
  network.add(Dense(20, activation="relu", use_bias=True, bias_initializer="glorot_normal", kernel_initializer="glorot_normal"))

  #Output layer that is fully connected with sigmoid activation
  network.add(Dense(1, activation="sigmoid", kernel_initializer="glorot_normal"))


  # Compile network model.
  network.compile(optimizer=get_optimizer(0.02919, 'Adagrad'),
               loss="binary_crossentropy",
               metrics=["binary_accuracy",
               tfa.metrics.F1Score(num_classes=2,
                                  average='weighted',
                                  threshold=0.5),
               tf.keras.metrics.AUC(from_logits=False)
                        ])


  #return completed network
  return network

In [ ]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(
                                 model=Neural_Net,
                                 epochs=125,
                                 batch_size=50,
                                 verbose=0,
                                 callbacks=[WandbCallback()]
                                 )

K_fold to ???????

In [ ]:
#Setup stratified Kfold for cross validation
seed = 123457
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

In [ ]:
#shows how good the model is
predictions = cross_val_predict(neural_network, X_train, y_train, cv=kfold)

ValueError: ignored

In [ ]:
conf_matrix = confusion_matrix(y_train, predictions)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Training Data Confusion Matirix with StratifiedKFold')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_train, predictions, average='binary')))
print('Precision: {}'.format(
    precision_score(y_train, predictions, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_train, predictions, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y_train, predictions, average='binary')))

Test data confrimation

In [ ]:
model = Neural_Net()
#Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=100, callbacks=[WandbCallback()])
test_pred = model.predict(X_test)


In [ ]:
predictions = (model.predict(X_train) > 0.5).astype("int32")

conf_matrix = confusion_matrix(y_train, predictions)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Training Data Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_train, predictions, average='binary')))
print('Precision: {}'.format(
    precision_score(y_train, predictions, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_train, predictions, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y_train, predictions, average='binary')))

In [ ]:
predictions_test = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
conf_matrix = confusion_matrix(y_test, predictions_test)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Test Data Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_test, predictions_test, average='binary')))
print('Precision: {}'.format(
    precision_score(y_test, predictions_test, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_test, predictions_test, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y_test, predictions_test, average='binary')))

In [ ]:
model.layers[0].get_weights()[1]

In [ ]:
#shows how good the model is
predictions = cross_val_predict(neural_network, X_test, y_test, cv=kfold)

In [ ]:
conf_matrix = confusion_matrix(y_test, predictions)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Test data Confusion Matirix with StratifiedKFold')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y_test, predictions, average='binary')))
print('Precision: {}'.format(
    precision_score(y_test, predictions, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y_test, predictions, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y_test, predictions, average='binary')))

---------------------------------------------------------------------
Further Testing with new datasets of real and sim data

In [ ]:
%cd drive/MyDrive/Zach/RealData
%ls

In [ ]:
df_real1 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Blue Order 2 (Ready4_ML).csv')
df1 = pd.read_csv('/content/drive/MyDrive/Zach/mergedata/OldData/blueSIM_test1.csv')

df_real1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
# Rename the 'B' column to 'C' in-place
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)

df2 = df_real1


df1['type'] = 0 #SimData
df2['type'] = 1 #RealData
df_combined = df1.append(df2)
df_combined

In [ ]:
df_combined['topics'] = df_combined['topics'].apply(lambda x: x.replace(" ", ""))
df_combined

In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df_combined['topics']= label_encoder.fit_transform(df_combined['topics'])
df_combined['msg']= label_encoder.fit_transform(df_combined['msg'])
df_combined['node']= label_encoder.fit_transform(df_combined['node'])
df_combined

In [ ]:
#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df_combined.drop(predict, axis=1)
y = df_combined[predict]
print(X.head())
print(y.head())

In [ ]:
predictions_combined = (model.predict(X) > 0.5).astype("int32")

In [ ]:
conf_matrix = confusion_matrix(y, predictions_combined)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Test data Confusion Matirix for Sim&Real data')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y, predictions_combined, average='binary')))
print('Precision: {}'.format(
    precision_score(y, predictions_combined, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y, predictions_combined, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y, predictions_combined, average='binary')))

-------------------------------------
Testing with only fake data

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Zach/mergedata/OldData/blueSIM_test1.csv')
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df1['type'] = 0 #SimData


df1['topics'] = df1['topics'].apply(lambda x: x.replace(" ", ""))
df1


# Encode labels in column 'species'.
df1['topics']= label_encoder.fit_transform(df1['topics'])
df1['msg']= label_encoder.fit_transform(df1['msg'])
df1['node']= label_encoder.fit_transform(df1['node'])
df1

#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df1.drop(predict, axis=1)
y = df1[predict]
print(X.head())
print(y.head())

predictions_df1 = (model.predict(X) > 0.5).astype("int32")

conf_matrix = confusion_matrix(y, predictions_df1)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Sim data ONLY Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y, predictions_df1, average='binary')))
print('Precision: {}'.format(
    precision_score(y, predictions_df1, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y, predictions_df1, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y, predictions_df1, average='binary')))

In [ ]:
predictions_df1

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Zach/RealData/Blue Order 2 (Ready4_ML).csv')
df1.rename(columns={'Run Time ': 'Run Time'}, inplace=True)
df1['type'] = 1 #RealData


df1['topics'] = df1['topics'].apply(lambda x: x.replace(" ", ""))
df1


# Encode labels in column 'species'.
df1['topics']= label_encoder.fit_transform(df1['topics'])
df1['msg']= label_encoder.fit_transform(df1['msg'])
df1['node']= label_encoder.fit_transform(df1['node'])
df1

#Setting our prediction target and dropping target columns from the main dataset
predict = 'type'
drops = ['Run Time', 'msg', 'node', 'topics']
X = df1.drop(predict, axis=1)
y = df1[predict]
print(X.head())
print(y.head())

predictions_df1 = (model.predict(X) > 0.5).astype("int32")

conf_matrix = confusion_matrix(y, predictions_df1)
print(conf_matrix, '\n')

conf_matrix_n = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_matrix_n, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Real data ONLY Confusion Matirix')
plt.xticks(rotation=45)
plt.show(block=False)

print('\nF1 Score: {}'.format(
    f1_score(y, predictions_df1, average='binary')))
print('Precision: {}'.format(
    precision_score(y, predictions_df1, average='binary'))) #weighted to show AUC-Precision/Recall curve
print('Recall: {}'.format(
    recall_score(y, predictions_df1, average='binary')))
print('Accuracy: {}'.format(
    accuracy_score(y, predictions_df1, average='binary')))

In [ ]:
predictions_df1